import files from imdb data, filter to desired results. From a brief look at the data I can see that 1, there are movies that haven't come out yet; 2, there are movies without genres. Since neither of these will be helpful for my analysis I have decided to remove them. Additionally my rationale has determined that I don't want to look at movies before 2014 so I've removed those too.

In [16]:
import pandas as pd
df_imdb_basics = pd.read_csv('zippedData/imdb.title.basics.csv.gz', compression='gzip')
df_filtered = df_imdb_basics[(df_imdb_basics['start_year'] <= 2020) & (df_imdb_basics['start_year'] >= 2014)]
df_imdb_ratings = pd.read_csv('zippedData/imdb.title.ratings.csv.gz', compression='gzip')
df_imdb_ratings = df_imdb_ratings[(df_imdb_ratings['numvotes'] > 100)]
df_imdb_akas = pd.read_csv('zippedData/imdb.title.akas.csv.gz', compression='gzip')

dropna values from genre because that will be a factor in analysis

In [17]:
df_filtered = df_filtered.dropna(subset=['genres'])

split genre out into multiple columns

In [18]:
genres = df_filtered['genres'].str.split(",", n = 1, expand = True)
df_filtered['genre1']= genres[0]
df_filtered['genre2']= genres[1]
genres2 = df_filtered['genre2'].str.split(",", n = 1, expand = True)
df_filtered['genre2']= genres2[0]
df_filtered['genre3']= genres2[1]
df_filtered.head()

,tconst,primary_title,original_title,start_year,runtime_minutes,genres,genre1,genre2,genre3
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama",Biography,Drama,None
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama,Drama,None,None
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama",Comedy,Drama,None
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy",Comedy,Drama,Fantasy
5,tt0111414,A Thin Life,A Thin Life,2018,75.0,Comedy,Comedy,None,None


In [19]:
from pandasql import sqldf
pysqldf =  lambda q: sqldf(q, globals())

In [20]:
q = '''SELECT *
        FROM df_filtered
        JOIN df_imdb_ratings
        USING(tconst)
        ;'''

imdb_joined_df = pysqldf(q)
imdb_joined_df.head()

,tconst,primary_title,original_title,start_year,runtime_minutes,genres,genre1,genre2,genre3,averagerating,numvotes
0,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama,Drama,None,None,6.9,4517
1,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy",Comedy,Drama,Fantasy,6.5,119
2,tt0137204,Joe Finds Grace,Joe Finds Grace,2017,83.0,"Adventure,Animation,Comedy",Adventure,Animation,Comedy,8.1,263
3,tt0315642,Wazir,Wazir,2016,103.0,"Action,Crime,Drama",Action,Crime,Drama,7.1,15378
4,tt0331314,Bunyan and Babe,Bunyan and Babe,2017,84.0,"Adventure,Animation,Comedy",Adventure,Animation,Comedy,5.0,302


In [25]:
q = '''SELECT *
        FROM imdb_joined_df
        JOIN df_imdb_akas
        ON df_imdb_akas.title_id = imdb_joined_df.tconst
        ;'''

imdb_all = pysqldf(q)
imdb_all.head()

,tconst,primary_title,original_title,start_year,runtime_minutes,genres,genre1,genre2,genre3,averagerating,numvotes,title_id,ordering,title,region,language,types,attributes,is_original_title
0,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama,Drama,None,None,6.9,4517,tt0069049,1,O Outro Lado do Vento,BR,None,imdbDisplay,None,0.0
1,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama,Drama,None,None,6.9,4517,tt0069049,2,The Other Side of the Wind,US,None,imdbDisplay,None,0.0
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama,Drama,None,None,6.9,4517,tt0069049,3,The Other Side of the Wind,GB,None,imdbDisplay,None,0.0
3,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama,Drama,None,None,6.9,4517,tt0069049,4,L'altra faccia del vento,IT,None,None,None,0.0
4,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama,Drama,None,None,6.9,4517,tt0069049,5,Другая сторона ветра,RU,None,None,None,0.0


In [34]:
imdb_all = imdb_all[imdb_all['region'] == 'US']
imdb_all = imdb_all.drop_duplicates(subset = 'tconst')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8011 entries, 1 to 103678
Data columns (total 19 columns):
tconst               8011 non-null object
primary_title        8011 non-null object
original_title       8011 non-null object
start_year           8011 non-null int64
runtime_minutes      7940 non-null float64
genres               8011 non-null object
genre1               8011 non-null object
genre2               5705 non-null object
genre3               3519 non-null object
averagerating        8011 non-null float64
numvotes             8011 non-null int64
title_id             8011 non-null object
ordering             8011 non-null int64
title                8011 non-null object
region               8011 non-null object
language             257 non-null object
types                2623 non-null object
attributes           775 non-null object
is_original_title    8011 non-null float64
dtypes: float64(3), int64(3), object(13)
memory usage: 1.2+ MB


In [43]:
q = '''SELECT *
        FROM imdb_all
        WHERE averagerating > 7.0
        ORDER BY averagerating DESC
        ;'''

top_rated = pysqldf(q)
top_rated.head()

,tconst,primary_title,original_title,start_year,runtime_minutes,genres,genre1,genre2,genre3,averagerating,numvotes,title_id,ordering,title,region,language,types,attributes,is_original_title
0,tt7131622,Once Upon a Time ... in Hollywood,Once Upon a Time ... in Hollywood,2019,159.0,"Comedy,Drama",Comedy,Drama,None,9.7,5600,tt7131622,20,Untitled #9,US,None,working,None,0.0
1,tt6842524,"Hare Krishna! The Mantra, the Movement and the...","Hare Krishna! The Mantra, the Movement and the...",2017,90.0,Documentary,Documentary,None,None,9.5,829,tt6842524,3,"Hare Krishna! The Mantra, the Movement and the...",US,None,None,None,0.0
2,tt8354112,Mosul,Mosul,2019,86.0,Documentary,Documentary,None,None,9.5,617,tt8354112,3,Mosul,US,None,imdbDisplay,None,0.0
3,tt6859280,"The Nagano Tapes: Rewound, Replayed & Reviewed","The Nagano Tapes: Rewound, Replayed & Reviewed",2018,73.0,Documentary,Documentary,None,None,9.4,192,tt6859280,3,"The Nagano Tapes: Rewound, Replayed & Reviewed",US,None,None,None,0.0
4,tt7738784,Peranbu,Peranbu,2018,147.0,Drama,Drama,None,None,9.4,9629,tt7738784,4,Peranbu,US,None,imdbDisplay,None,0.0


In [42]:
#rating by runtime

In [ ]:
#rating by genre
imdb_all